In [4]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from langchain_anthropic import ChatAnthropic
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


load_dotenv(override=True)

MODEL = "gpt-4.1-nano"

DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "all-MiniLM-L6-v2"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500

# openai = OpenAI()

In [5]:
# Inspired by LangChain's Document - let's have something similar

class Result(BaseModel):
    page_content: str
    metadata: dict

In [6]:
# A class to perfectly represent a chunk

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

In [ ]:
def fetch_documents():
    """A homemade version of the LangChain DirectoryLoader"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [ ]:
documents = fetch_documents()

In [52]:
def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

In [ ]:
print(make_prompt(documents[0]))

In [54]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [55]:
def process_document(document):
    """Process a document by splitting it into overlapping chunks using claude-haiku-4-5 via LangChain."""
    llm = ChatAnthropic(model="claude-haiku-4-5")
    structured_llm = llm.with_structured_output(Chunks)
    messages = make_messages(document)
    doc_as_chunks_obj = structured_llm.invoke(messages)
    return [chunk.as_result(document) for chunk in doc_as_chunks_obj.chunks]

In [ ]:
print(process_document(documents[0]))	

In [57]:
from tqdm import tqdm

def create_chunks(documents):
    limit= 3
    chunks = []
    docs_to_process = documents[:limit] if limit is not None else documents
    for doc in tqdm(docs_to_process):
        chunks.extend(process_document(doc))
    return chunks

In [ ]:
chunks = create_chunks(documents)

In [ ]:
print(chunks)

In [60]:
from langchain_huggingface import HuggingFaceEmbeddings

def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectors = embeddings.embed_documents(texts)

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

In [ ]:
create_embeddings(chunks)

In [62]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [ ]:
# visualizing the vectors in 2D
tsne = TSNE(n_components=2, random_state=42, perplexity=10)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
# visualizing the vectors in 3D
tsne = TSNE(n_components=3, random_state=42, perplexity=10)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

In [65]:
class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [72]:
import json
from typing import List

from langchain_anthropic import ChatAnthropic
from langchain_core.documents import Document
from langchain_core.messages import SystemMessage, HumanMessage


llm = ChatAnthropic(
    model="claude-haiku-4-5",
    temperature=0.0,
)


def rerank(question: str, chunks: List[Document]) -> List[Document]:
    """
    Re-rank retrieved document chunks by relevance to the question using Claude.
    """
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance,
but you may be able to improve on that.

You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.

Reply ONLY with valid JSON in the following format:
{"order": [<chunk_id_1>, <chunk_id_2>, ...]}

Include all the chunk ids you are provided with, reranked.
"""

    user_prompt_lines = [
        f"The user has asked the following question:\n\n{question}\n",
        "Order all the chunks of text by relevance to the question, from most relevant to least relevant.",
        "Here are the chunks:\n",
    ]

    for index, chunk in enumerate(chunks, start=1):
        user_prompt_lines.append(
            f"# CHUNK ID: {index}\n\n{chunk.page_content}\n"
        )

    user_prompt = "\n".join(user_prompt_lines)

    messages = [
        SystemMessage(content=system_prompt.strip()),
        HumanMessage(content=user_prompt.strip()),
    ]

    structured_llm = llm.with_structured_output(RankOrder)
    rank_order = structured_llm.invoke(messages)
    order = rank_order.order
    print(order)
    return [chunks[i - 1] for i in order]

In [67]:
from langchain_huggingface import HuggingFaceEmbeddings

RETRIEVAL_K = 10

def fetch_context_unranked(question):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    query = embeddings.embed_query(question)
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    return chunks

In [69]:
question = "Overview of Insurellm?"
chunks = fetch_context_unranked(question)

In [ ]:
for chunk in chunks:
    print(chunk.page_content[:100]+"...")

In [ ]:

reranked = rerank(question, chunks)


In [ ]:
for chunk in reranked:
    print(chunk.page_content[:15]+"...")

In [75]:
question = "Who went to Manchester University?"
RETRIEVAL_K = 20
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [ ]:
reranked = rerank(question, chunks)

In [77]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

In [ ]:
reranked[0].page_content

In [79]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [80]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [81]:
def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [95]:
from typing import List, Optional

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage


# You can change the model name here
llm = ChatAnthropic(
    model="claude-haiku-4-5",   # or claude-3-haiku-20240307, claude-3-opus, etc.
    temperature=0.0,
    max_tokens=60,
)


def rewrite_query(question: str, history: Optional[List[dict]] = None) -> str:
    """
    Rewrite the user's question to be a more specific question that is more likely
    to surface relevant content in the Knowledge Base.
    """
    if history is None:
        history = []

    # Format history exactly like your original string interpolation expected
    # (assuming history is list of dicts with "role" and "content")
    history_str = ""
    if history:
        lines = []
        for msg in history:
            role = "User" if msg.get("role", "").lower() == "user" else "Assistant"
            content = msg.get("content", "").strip()
            if content:
                lines.append(f"{role}: {content}")
        history_str = "\n".join(lines)
    else:
        history_str = "(no history)"

    # ──────────────────────────────────────────────
    # Your **exact original prompt** — unchanged
    # ──────────────────────────────────────────────
    system_content = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history_str}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""

    messages = [
        SystemMessage(content=system_content),
        HumanMessage(content="Please rewrite now."),
    ]

    # Call Claude
    response = llm.invoke(messages)

    # Claude returns a message object → extract content
    query = response.content.strip()

    # Very light post-processing (in case model adds subtle prefix despite instruction)
    for bad_start in ["Query:", "Search:", '"', "Refined query:", "Here is the query:"]:
        if query.startswith(bad_start):
            query = query[len(bad_start):].strip()

    query = query.strip('"').strip()

    return query

In [ ]:
print(rewrite_query("Tell me about the Insurellm?"))

In [100]:
from typing import List, Tuple

from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage

# Configure Anthropic LLM (adjust model / parameters as needed)
llm = ChatAnthropic(
    model="claude-haiku-4-5",   # or "claude-3-haiku-20240307", etc.
    temperature=0.0,
    max_tokens=2000,
)


def answer_question(
    question: str,
    history: List[dict] = None,
) -> Tuple[str, List]:
    """
    Answer a question using RAG and return the answer and the retrieved context.
    Uses Anthropic Claude for the generation step.
    """
    if history is None:
        history = []

    # 1. Rewrite the query (assuming rewrite_query is already using Anthropic)
    query = rewrite_query(question, history)
    print(query)  # for debugging / logging

    # 2. Retrieve chunks (unchanged)
    chunks = fetch_context(query)

    # 3. Build messages for RAG
    messages: List[BaseMessage] = []

    # Convert conversation history to LangChain message objects
    for msg in history:
        role = msg.get("role", "").lower()
        content = msg.get("content", "")
        if role == "user":
            messages.append(HumanMessage(content=content))
        elif role == "assistant":
            messages.append(AIMessage(content=content))

    # Add current question + RAG context
    # (You can replace make_rag_messages with your actual logic)
    # Here is a minimal version — adjust according to your real make_rag_messages

    context_text = "\n\n".join(
        f"[Document {i+1}]\n{chunk.page_content if hasattr(chunk, 'page_content') else str(chunk)}"
        for i, chunk in enumerate(chunks)
    )

    rag_prompt = f"""You are a helpful assistant answering questions about Insurellm using the provided context.
Use only the information in the context below to answer. If the context doesn't contain the answer,
say you don't know or that the information is not available.

Context:
{context_text}

Current question:
{question}
"""

    messages.append(HumanMessage(content=rag_prompt))

    # 4. Call Anthropic Claude
    response = llm.invoke(messages)

    # 5. Extract the answer
    answer = response.content.strip()

    return answer, chunks

In [ ]:
answer_question("what are Insurellm are 5 core insurance?", [])

In [ ]:
answer_question("Who went to Manchester University?", [])